In [1]:
!pip install Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.3 MB/s eta 0:00:00


In [2]:
import math
from collections import Counter
import re

def build_trigram_model(corpus):
    trigram_counter = Counter()
    for sentence in corpus:
        # Tách từ, làm sạch câu
        tokens = re.findall(r'\w+', sentence.lower())
        tokens = ['<s>'] + tokens + ['</s>']  # Thêm token bắt đầu/kết thúc câu

        for i in range(len(tokens) - 2):
            trigram = (tokens[i], tokens[i+1], tokens[i+2])
            trigram_counter[trigram] += 1

    return trigram_counter

In [3]:
import pandas as pd

df = pd.read_csv("/kaggle/input/luat123/error_dataset.csv")

corpus = df['ground_truth']
len(corpus)

100000

In [4]:
trigram_counter = build_trigram_model(corpus)

In [5]:
# def compute_trigram_score(left_context, candidate_token, right_context, trigram_counter):
#     # left_context, right_context là list token, ví dụ: ["tôi", "thích"]
#     w1 = left_context[-2] if len(left_context) >= 2 else '<s>'
#     w2 = left_context[-1] if len(left_context) >= 1 else '<s>'
#     w3 = candidate_token

#     trigram = (w1, w2, w3)
#     bigram = (w1, w2)

#     # Add-1 smoothing
#     trigram_count = trigram_counter.get(trigram, 0) + 1
#     bigram_count = sum(count for (a, b, _), count in trigram_counter.items() if a == w1 and b == w2) + len(trigram_counter)

#     # Log-prob để tránh underflow
#     return math.log(trigram_count / bigram_count)

# # Ví dụ chọn candidate tốt nhất:
# left_context = ["tôi", "thích"]
# candidates = ["học", "ăn", "chơi"]

# best_pred = max(candidates, key=lambda x: compute_trigram_score(left_context, x, [], trigram_counter))
# print("Best Prediction:", best_pred)


In [6]:
def compute_trigram_score(left_context, candidate_token, trigram_counter):
    w1 = left_context[-2] if len(left_context) >= 2 else '<s>'
    w2 = left_context[-1] if len(left_context) >= 1 else '<s>'
    w3 = candidate_token

    trigram = (w1, w2, w3)
    bigram_count = sum(count for (a, b, _), count in trigram_counter.items() if a == w1 and b == w2) + 1

    trigram_count = trigram_counter.get(trigram, 0) + 1

    return math.log(trigram_count / bigram_count)


In [7]:
def detect_error(model, tokenizer, tokens, max_len=128, device='cuda'):
    model.eval()
    model.to(device)

    # Tokenize trước để lấy word_ids
    encoding = tokenizer(tokens,
                         is_split_into_words=True,
                         padding="max_length",
                         truncation=True,
                         max_length=max_len,
                         return_tensors='pt'
                         #return_tensors=None
                        ) 
    word_ids = encoding.word_ids()
    encoding = {k: v.to(device) for k, v in encoding.items()} # trả về tensor cuda và mask cuda
    

    # prediction
    with torch.no_grad():  # tắt gradient
        outputs = model(**encoding) # gọi forward pass của model với đầu vào là encoding
        # logits là raw score, output của model
        logits = outputs.logits # (batch_size, seq_len, num_labels)
        preds = torch.argmax(logits, dim=-1)[0].cpu().numpy() # cần phải đưa về cpu thì ms gọi numpy đc

    # Print ra đúng định dạng của input
    result = [0] * len(tokens)
    prev_word_idx = None
    for token_idx, word_idx in enumerate(word_ids):
        if word_idx is None:
            continue
        if word_idx != prev_word_idx:
            result[word_idx] = preds[token_idx]
            prev_word_idx = word_idx

    return [x for x in result]

In [8]:
import torch
from Levenshtein import distance as levenshtein_distance
import math

def normalize_trigram(score, min_score=-10, max_score=0):
    return (score - min_score) / (max_score - min_score) if max_score != min_score else 0.0


def correct_error_encoder_top_k(model, tokenizer, corrupted_tokens, error_flags,
                                trigram_counter, top_k=10, max_len=64, device='cuda',
                                confidence_threshold=0.8, word_length_threshold=5, 
                                trigram_weight=0.6, levenshtein_weight=0.4):

    assert len(corrupted_tokens) == len(error_flags), "Unmatch Length!"
    model.eval()
    model.to(device)

    masked_tokens = [
        tokenizer.mask_token if flag else tok
        for tok, flag in zip(corrupted_tokens, error_flags)
    ]
    masked_text = " ".join(masked_tokens)

    inputs = tokenizer(masked_text, return_tensors="pt", max_length=max_len, 
                       padding="max_length", truncation=True, is_split_into_words=False).to(device)

    word_ids = inputs.word_ids()
    input_ids = inputs["input_ids"][0]

    with torch.no_grad():
        logits = model(**inputs).logits  # (1, seq_len, vocab_size)

    printed = set()
    replaced_tokens = corrupted_tokens.copy()

    for token_idx, word_idx in enumerate(word_ids):
        if word_idx is None or word_idx in printed or error_flags[word_idx] != 1:
            continue
        printed.add(word_idx)

        token_logits = logits[0, token_idx]
        probs = torch.softmax(token_logits, dim=-1)
        topk_ids = torch.topk(probs, top_k).indices.tolist()
        topk_tokens = [tokenizer.convert_ids_to_tokens(tid).lstrip("▁") for tid in topk_ids]
        topk_confs = [probs[tid].item() for tid in topk_ids]

        model_confidence = topk_confs[0]
        left_context = replaced_tokens[:word_idx]

        print("model confidence: ", model_confidence)
        if model_confidence > confidence_threshold:
            print("Choose method 1")
            best_pred = topk_tokens[0]
        elif len(corrupted_tokens[word_idx]) < word_length_threshold:
            print("Choose method 2")
            best_pred = max(topk_tokens, key=lambda x: compute_trigram_score(left_context, x, trigram_counter))
        else:
            print("Choose method 3")
            best_score = -math.inf
            for cand in topk_tokens:
                trigram_s = compute_trigram_score(left_context, cand, trigram_counter)
                trigram_s_norm = normalize_trigram(trigram_s)
                lev_sim = 1 - levenshtein_distance(corrupted_tokens[word_idx], cand) / max(len(corrupted_tokens[word_idx]), len(cand))
                combined_s = trigram_weight * trigram_s_norm + levenshtein_weight * lev_sim
                if combined_s > best_score:
                    best_score = combined_s
                    best_pred = cand

        replaced_tokens[word_idx] = best_pred

    return " ".join(replaced_tokens)


In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSeq2SeqLM, AutoModelForMaskedLM
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import f1_score
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
checkpoint_path = "/kaggle/input/train-token-classification-roberta/checkpoint/best"
tokenizer_detect = AutoTokenizer.from_pretrained(checkpoint_path, use_fast=True)
model_detect = AutoModelForTokenClassification.from_pretrained(checkpoint_path)
model_detect.to(device)

#####
model_name_2 = "xlm-roberta-base"
#model_name_2 = "/kaggle/input/mlm-encoder/mlm_best_model"
tokenizer_correct_en = AutoTokenizer.from_pretrained(model_name_2)
model_correct_en = AutoModelForMaskedLM.from_pretrained(model_name_2)

2025-05-13 11:27:17.646790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747135637.829540      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747135637.882735      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
sentence = 'Đặc biệt việc này cũng giúp tạo vị tFế cạnh tranh cho các kế TOÁN viên vrà tránh nguy cơ bị đào thải'

error_flag = detect_error(model_detect, tokenizer_detect, sentence.split())
print("Prediction mask:   ", error_flag)

Prediction mask:    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]


In [12]:
corrected_sentence = correct_error_encoder_top_k(model_correct_en, tokenizer_correct_en, sentence.split(),error_flag, trigram_counter )
print("prediction sentence:  ", corrected_sentence)

model confidence:  0.7780030369758606
Choose method 2
model confidence:  0.9470243453979492
Choose method 1
model confidence:  0.5837796926498413
Choose method 2
prediction sentence:   Đặc biệt việc này cũng giúp tạo vị thế cạnh tranh cho các kế toán viên và tránh nguy cơ bị đào thải
